<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# Google GenAI SDK - Building a Sequential Agent

## Install Dependencies

In [ ]:
!pip install -q google-genai arize-phoenix-otel openinference-instrumentation-google-genai

## Connect to Phoenix

In [ ]:
import os
from getpass import getpass

from google import genai

from phoenix.otel import register

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

tracer_provider = register(auto_instrument=True, project_name="google-genai-sequential-agent")
tracer = tracer_provider.get_tracer(__name__)

## Authenticate with Google Vertex AI

In [ ]:
!gcloud auth login

In [ ]:
# Create a client using the Vertex AI API, you could also use the Google GenAI API instead here
client = genai.Client(vertexai=True, project="sandbox-455622", location="us-central1")

# Sequential Agent

In [ ]:
# --- 1. Define helper methods ---


def get_model(model_name):
    """Instantiate and return a model."""
    return model_name


def generate_story(client, model, user_input):
    """Generate a short story based on user input."""
    story_prompt = (
        "You are a creative short story writer.\n"
        "Write a brief, engaging story (3–4 paragraphs) about an unexpected "
        "adventure.\n"
        "Be imaginative but concise.\n"
        "User Input: "
    )
    story_response = client.models.generate_content(
        model=model,
        contents=story_prompt + user_input,
    )
    return story_response.text.strip()


def critique_story(client, model, story):
    """Analyze the provided story and provide critique."""
    critic_prompt = (
        "You are a literary critic.\n"
        "Analyze the provided story for its strengths and weaknesses.\n"
        "Comment on plot, characters, and overall impact.\n"
        "Provide 2–3 specific suggestions for improvement.\n\n"
        f"Story:\n{story}"
    )
    critic_response = client.models.generate_content(model=model, contents=critic_prompt)
    return critic_response.text.strip()


def improve_story(client, model, original_story, critique):
    """Revise the story based on the critique."""
    revision_prompt = (
        "You are a creative short story writer.\n"
        "Revise the following story based on the critique provided.\n\n"
        f"Critique:\n{critique}\n\n"
        f"Original Story:\n{original_story}\n\n"
        "Please produce an improved version of the story, addressing the suggestions.\n"
    )
    revision_response = client.models.generate_content(model=model, contents=revision_prompt)
    return revision_response.text.strip()


# --- 2. Main execution ---
@tracer.agent()
def run_agent(user_input):
    # Instantiate the models
    story_model = get_model("gemini-2.0-flash-001")
    critic_model = get_model("gemini-2.0-flash-001")

    # Generate the initial story
    generated_story = generate_story(client, story_model, user_input)

    # Critique the generated story
    story_critique = critique_story(client, critic_model, generated_story)

    # Display the results
    print("=== Generated Story ===\n")
    print(generated_story)
    print("\n=== Critique ===\n")
    print(story_critique)

    # Improve the story based on the critique
    improved_story = improve_story(client, story_model, generated_story, story_critique)

    # Display the improved story
    print("\n=== Improved Story ===\n")
    print(improved_story)
    return improved_story


run_agent(user_input=input("Please enter a prompt or seed for your story: "))